In [44]:
import numpy as np
import skimage as ski
from skimage import io
import matplotlib.pyplot as plt
import matplotlib.image as mpltimg
import os
import cv2
from PIL import Image
import keras as krs
import tensorflow as tf

In [45]:
#trainin stuffs
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import conv_utils

In [46]:
dataset = []
labels = []
# importing the dataset first for train/test 
# labels are binary (1, 0)
# 1 stands for infected
# 0 stands for control

In [47]:
# here goes the infected batch of data
infected_imgs = os.listdir('E:/DEMO/data_set/infected')
for i, image_name in enumerate(infected_imgs): 
    image = cv2.imread("E:/DEMO/data_set/infected/" +  image_name)
    image = Image.fromarray(image, 'RGB')
    image = image.resize((100, 100))
    dataset.append(np.array(image))
    labels.append(1)

In [48]:
# here goes the control batch of data
infected_imgs = os.listdir('E:/DEMO/data_set/control/')
for i, image_name in enumerate(infected_imgs):   
    image = cv2.imread('E:/DEMO/data_set/control/' + image_name)
    image = Image.fromarray(image, 'RGB')
    image = image.resize((100, 100))
    dataset.append(np.array(image))
    labels.append(0)

In [49]:
# converting dataset and labels from lists to np.ndarray() for the upcoming work
dataset = np.array(dataset)
labels = np.array(labels)

In [50]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataset, labels, test_size = 0.20, random_state = 0)

In [51]:
x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1)

In [ ]:
#creatin za model
model = Sequential()
